# Demo for paper "First Order Motion Model for Image Animation"

**Clone repository**

In [ ]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 85, done.
remote: Total 85 (delta 0), reused 0 (delta 0), pack-reused 85
Unpacking objects: 100% (85/85), done.


In [3]:
ls

 Volume in drive C has no label.
 Volume Serial Number is FE19-9A30

 Directory of C:\Users\anton\first-order-model

12/08/2020  21:51    <DIR>          .
12/08/2020  21:51    <DIR>          ..
11/08/2020  22:39                26 .dockerignore
11/08/2020  22:39                30 .gitignore
12/08/2020  19:41    <DIR>          .ipynb_checkpoints
12/08/2020  01:33    <DIR>          __pycache__
11/08/2020  22:39             4,162 animate.py
11/08/2020  22:39            12,892 augmentation.py
11/08/2020  22:39    <DIR>          config
11/08/2020  22:39             6,033 crop-video.py
12/08/2020  19:57           788,997 damedane.psd
12/08/2020  20:58    <DIR>          damedane_new
11/08/2020  22:39    <DIR>          data
11/08/2020  22:39         1,536,874 demo.ipynb
11/08/2020  22:39             6,887 demo.py
11/08/2020  22:39               484 Dockerfile
12/08/2020  21:51         2,396,416 first_order_local.ipynb
11/08/2020  22:39             7,224 frames_dataset.py
12/08/2020  21:42    <D

**Original source folder https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw?usp=sharing**

**Load driving video and source image**

In [7]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

inputName = "umer"
drivingVideo = "damefinal.gif"

source_image = imageio.imread(f"./inputs/{inputName}.png")
driving_video = imageio.mimread(f"./inputs/{drivingVideo}")


#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    
print("Done!")
# HTML(display(source_image, driving_video).to_html5_video())

Done!


**Create a model and load checkpoints**

In [8]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='./vox-cpk.pth.tar')
print("Done!")

Done!


**Perform image animation**

In [ ]:
from demo import make_animation
from skimage import img_as_ubyte
import os

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

outputName="umerdamedane"

#save resulting video
imageio.mimsave(f"./outputs/{outputName}.mp4", [img_as_ubyte(frame) for frame in predictions], fps=30)
os.system(f"ffmpeg -i outputs/{outputName}.mp4 -i inputs/damedane.mp4 -vcodec copy -acodec copy final/{outputName}.mp4")
print("Video Complete!")


HTML(display(source_image, driving_video, predictions).to_html5_video())

**In the cell above we use relative keypoint displacement to animate the objects. We can use absolute coordinates instead,  but in this way all the object proporions will be inherited from the driving video. For example Putin haircut will be extended to match Trump haircut.**

In [12]:
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True)
HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:17<00:00, 28.18it/s]
